## Developer building_id bug

Sam Maurer, August 2015

In [2]:
%load_ext autoreload
%autoreload 2
if 'sim' not in globals():
    import os; os.chdir('..')
import models
import datasources
import pandas as pd
import numpy as np
import urbansim.sim.simulation as sim
import warnings 
warnings.filterwarnings("ignore", category=FutureWarning)  # triggered by hh transition model

### Execute prior models

In [3]:
sim.run([
    "neighborhood_vars",

    "rsh_simulate",
    "rrh_simulate",

    "households_relocation_filtered",
    "households_transition",
    
    "hlcm_owner_simulate",
    "hlcm_renter_simulate",

    "price_vars",
    "cap_rate_precompute",

    "feasibility",
    "residential_developer",
        
], years=[2010])

Running year 2010
Running model 'neighborhood_vars'
Computing accessibility variables
Computing sum_residential_units
Filling column building_type_id with value 1 (0 values)
Filling column residential_units with value 0 (0 values)
Filling column year_built with value 1966.0 (0 values)
Filling column non_residential_sqft with value 0 (0 values)
Filling column shape_area with value 604.336158108 (0 values)
Filling column zone_id with value 1178 (0 values)
Removed 4 rows because they contain missing values
Computing sum_nonresidential_units
Removed 4 rows because they contain missing values
Computing ave_sqft_per_unit
Removed 2 rows because they contain missing values
Computing ave_lot_size_per_unit
Removed 2 rows because they contain missing values
Computing population
Removed 79699 rows because they contain missing values
Computing poor
Removed 53966 rows because they contain missing values
Computing renters
Removed 4919 rows because they contain missing values
Computing sfdu
Removed 1 

/Users/smmaurer/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1164: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


### Demonstration of building_id bug

In [7]:
buildings = sim.get_table("buildings")
b = buildings.to_frame()

units = sim.get_table("residential_units")
u = units.to_frame()

In [12]:
# building_id is the index of the buildings table
b = b.reset_index()

In [14]:
u.building_id[~u.building_id.isin(b.building_id)].describe()

count       3426.000000
mean      862802.791302
std       548494.095740
min            0.000000
25%       178807.500000
50%       953746.000000
75%      1256184.250000
max      1841217.000000
Name: building_id, dtype: float64

In [16]:
# ??? maybe there's no bug
# try running this before and after the developer model

Here's a draft of a Github issue:

Bug where utils.run_developer() creates new residential units

I think I ran into a bug where `utils.run_developer()` creates new residential units. 

It assigns the new units a `building_id` based on the index of the `new_buildings` dataframe (see [lines 810-12](https://github.com/UDST/urbansim_defaults/blob/master/urbansim_defaults/utils.py#L810-L812)), but when those buildings are added to the `buildings` table, their index is actually shifted by the maximum previously existing `building_id` (see [line 796](https://github.com/UDST/urbansim_defaults/blob/master/urbansim_defaults/utils.py#L817), and the [Developer.merge()](https://github.com/UDST/urbansim/blob/master/urbansim/developer/developer.py#L211-L235) method).  

Here's a notebook that demonstrates the resulting mis-match: <tk>

I think we can fix it by just doing the same index shift when we assign `building_id` to new units, right? I'm trying this out in a fork but wanted to get your input. Thanks!